# 增量式的平滑与建图

前面我们讨论了离线的平滑与建图的方法，即所有数据都提前采集好，然后进行Batch optimization。这章我们讨论增量式的系统，即观测作为一个时间序列到来。除此之外，当一个场景过于庞大以至于不能进行Batch优化时，也需采用增量式优化的技巧降低计算量(深度学习不能进行Batch优化，采用Mini-Batch优化)。增量式计算技术的本质是要复用已经计算的结果。

## 一、更新矩阵分解
在增量式优化的设定中，我们想要用新的观测更新最近的矩阵分解结果，来复用之前观测的计算结果。假设原本的线性系统是$[A|b]$，在计算QR分解后得
<div align="center"><img src="../resources/chapter5/Ab_QR.png" width=30%></div>

则原本得线性最小二乘优化问题$||A\Delta-b||_2^2=||R\Delta - d|| + c$。下面考虑一个新的观测到达(构建了一个新的因子)，我们需要一种技术可以复用之前QR分解的结果，具体如下所示。
<div align="center"><img src="../resources/chapter5/Ab_incremental_QR.png" width=30%></div>

其本质上类似Householder变换$G = I - ww^{T}$，在一个分块矩阵中令分块为0。
<div align="center"><img src="../resources/chapter5/incremental_elimination.png" width=30%></div>

## 二、Bayes树
当使用消元算法把因子图变成贝叶斯网络时，贝叶斯网络有一个重要的性质：它是一个**弦图**(chordal graph),即任何大于3个节点构成的无向环必须有一条弦(类似三角剖分)。

由于Bayes网络是一张弦图，因此它里面的**团(clique)组成的节点可以构成一棵树，称为Bayes树**。团(clique)在图论中指可以全连接子图。若所有团构成无向树称为团树，Bayes树是一种有向树，有向边和贝叶斯网络的含义相同，代表conditioning（从原因指向结果）。

<div align="center"><img src="../resources/chapter5/bayes_tree.png" width=30%></div>

构建Bayes树的一个优势是在加入新的因子时，可以只更新部分Bayes树（从因子更新的节点，一直到根节点）。

<div align="center"><img src="../resources/chapter5/bayes_tree_update.png" width=30%></div>

